In [13]:
# Importation des librairies, environnement qt pour avoir des fenêtres flottantes
%matplotlib qt 
import numpy as np
from skimage import io, exposure, morphology
from skimage.filters import threshold_local
from skimage.measure import regionprops
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import matplotlib.pyplot as plt

In [4]:
#---------------------FONCTIONS AUXILIAIRES
# Génération de masque circulaire à partir d'une image, d'un centre et d'un rayon
def genere_mask (image, centre, rayon):
    shape = np.shape(image)
    basex = np.arange(0,shape[0]); basey = np.arange(0, shape[1])
    [i0,j0] = centre
    mask = (basex[:,np.newaxis] - i0)**2 + (basey[np.newaxis,:]-j0)**2 < rayon**2
    return mask

# Détourage carré d'une image 
def crop_image (image,centre, rayon):
    [i0,j0] = centre
    image_cropped = image[int(i0 - rayon): int(i0 + rayon), int(j0 - rayon): int(j0 + rayon)]
    return image_cropped

# Checker si une valeur est comprise dans un intervalle
def est_compris_entre(bbox_area,aire1,aire2):
    aire_max = max(aire1,aire2);
    aire_min = min(aire1,aire2);
    return (bbox_area <= aire_max and bbox_area >= aire_min)

# Normalisation de l'intensité d'une image entre 0 et 1
def normalise(img):
    norm = (img - np.min(img)) / (np.max(img) - np.min(img))
    return norm

In [5]:
#PARAMETRES GLOBAUX
# Paramètre gamma pour la correction gamma
gamma = 3

# Taille de la fenêtre pour le seuillage local
block_size_threshold = 17

# Solidité minimale des particules (voir "solidity" dans la documentation Python ou dans le coeur du manuscrit)
solidity_min = 0.87

# Différence grand axe, petit axe d'une région
dxymax = 4

# Tailles (aires) minimales des petites et des grandes particules
# Petites particules
p_min_area = 9*10 
p_max_area = 13*13

# Grandes particules
g_min_area = 13*14
g_max_area = 16*16

# Distance minimale inter-particules
dmin = 19

In [6]:
# Lecture de l'image
image = io.imread(r'.\image.tif', as_gray = True)

# Chargement du centre et du rayon du domaine circulaire central
rmax = np.loadtxt(r'.\rmax.txt');
centre = np.loadtxt(r'.\centre.txt')

In [7]:
# Normalisation de l'image
image_normalise = normalise(image)

# Création d'un masque sur les régions non-utiles (extérieur du domaine circulaire central)
mask = genere_mask(image_normalise,centre,rmax)
image_normalise[~mask] = 1

# Détourage de l'image
image_detoure = crop_image(image_normalise, centre,rmax)

# Amélioration du contraste, ajustement gamma
image_gamma = exposure.adjust_gamma(image_detoure,gamma)

# Matrice de seuils locaux
seuil_local = threshold_local(image_gamma, block_size_threshold)

# Binarisation de l'image
image_binaire = image_gamma > seuil_local

In [8]:
# Technique du watershed pour créer des bassins, voir mes autres articles pour l'explication pas-à-pas
distance = ndi.distance_transform_edt(image_binaire)
l_min = morphology.local_minima(-distance)
markers, _ = ndi.label(l_min)
labels = watershed(-distance, markers = markers, mask=image_binaire)
labels, _ = ndi.label(labels)


# Récupération des propriétés des régions labellisées créées par le watershed
regions = regionprops(labels)

In [9]:
# Détection des particules
labels_number = np.arange(1,np.max(labels)+1) # Récupération des numéros de toutes les régions labellisées
# la liste "taille" va stocker les tailles pour chaque particule, petite : 1, grande : 2
taille = []; 
# la liste "x" stocke la coordonnée x des particules
x = []; 
# la liste "y" stocke la coordonnée y des particules
y = []; 
# la liste 's' stocke la solidité des particules
s = [];


#Première sélection basée sur les attributs géométriques
for label in labels_number :
    region = [region for region in regions if region.label == label][0]

    solidity = region.solidity # solidité de la région 
    centroid = region.centroid # coordonnées du centre de la région
    b = region.bbox # coordonnées des coins du rectangle circonscrit à la région
    b_area = region.bbox_area # aire du rectangle circonscrit à la région
    dxy = abs(abs(b[2] - b[0]) - abs(b[3] - b[1])) # différence entre le grand et le petit axe de la région


    if(solidity >= solidity_min and dxy <dxymax ): # Elimination des formes non circulaires
        if (est_compris_entre(b_area, p_min_area, p_max_area)): # Récupération petites particules
            x.append(centroid[1]); y.append(centroid[0])
            s.append(solidity)
            taille.append(1) # 1 = petite particule
        else :
            if (est_compris_entre(b_area, g_min_area, g_max_area)): # Récupération grosses particules
                x.append(centroid[1]); y.append(centroid[0])
                s.append(solidity)
                taille.append(2) # 2 = grande particule

In [10]:
# Deuxième sélection basée sur les distances inter-particules  
# Cette partie est un raffinement, sur laquelle je pourrais écrire un article au besoin ;) 
count = np.zeros((len(x),1))
for i in range(len(x) - 1):
    for j in range(i+1,len(x)):
        dist = np.sqrt((x[i] - x[j])**2 + (y[i] - y[j])**2)
        if (dist < dmin):
            count[i] +=1;
            count[j] +=1;

# Au sortir de cette étape, le vecteur "count" contient pour chaque particule le nombre de fois où la distance avec les voisins 
#-est inférieure à la distance inter-particules minimale, ce qui permet d'éliminer certaines régions
        for k in [i,j]:
            # condition d'élimination (on rajoute un critère sur la solidité pour plus de sécurité)
            if count[k] >= 2 and s[k] < 0.95: # Donc forcément vide interstitiel 
                taille[k] = 0

In [11]:
#Reconditionnment pour enregistrement : on transforme toutes les listes en vecteurs numpy
taille = np.asarray(taille)
x = np.asarray(x);
y = np.asarray(y);
s = np.asarray(s)

# On ne conserve logiquement que les régions qui ont été catégorisées (soit 1 ou 2)
x = x[taille != 0];   
y = y[taille != 0];
s = s[taille != 0];
taille = taille[taille !=0];

In [14]:
# Affichage
# Récupération des coordonnées des petites particules
xp = x[taille == 1]; yp = y[taille == 1]

# Récupération des coordonnées des grandes particules
xg = x[taille == 2]; yg = y[taille == 2]

# Figure
plt.figure()
plt.imshow(image_detoure, cmap = 'gray')
pt = 15
s = [pt] * len(xp)
c =['r']* len(xp)
plt.scatter(xp, yp, s = s, c= c)

s = [pt] * len(xg)
c =['b']* len(xg)
plt.scatter(xg,yg, s = s, c= c)
plt.gca().set_aspect('equal')

plt.axis('off')

(-0.5, 1041.5, 1041.5, -0.5)